In [ ]:
using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using DelimitedFiles





function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 outgoing and 5 incoming connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,Splitting_Strength)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end
    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end





function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    
    
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return (u[t+1,:] - u[t,:]).*0.5

        else
            return (u[t-1,:] - u[t,:]).*0.5
        end
    else
        if t==1
            return u[t+2,:] - 2*u[t+1,:]  + u[t,:]
        elseif t==2^Splits
            return u[t-2,:] - 2*u[t-1,:] + u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits
        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function Epigenetic_Evolution_No_Epi(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]

    #⊗_(Wi, bi) = Wi .* bi
    
    for t=1:2^Splits
        du[t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[t,:].+CArray[:])) .- u[t,:] +KArray.*Gradient(u[:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end


function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du .= Noise
    
end



function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end






function Epi_Plotter(Indices,K,Final_Epigenetic_Factors,Generation)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".1") #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".2") #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".3") #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".4") #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".5") #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".6") #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".7") #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10,title="Generation: " *string(Generation) *".8") #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end


function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)

    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])                
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)
    elseif Diffusing_System

        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    
    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        sol = solve(prob,LambaEulerHeun())
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])                
    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection,PositionArray[1,:,:]
end
        
        
function Running_Network_No_Epi(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)

    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution_No_Epi, σ_Genes,PositionArray[1,:,:],tspan,Parameters)
        else
            prob = ODEProblem(Epigenetic_Evolution_No_Epi,PositionArray[1,:,:],tspan,Parameters)
        end    #Solving
                
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,LambaEulerHeun())
        end
                    
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray[1,:,:] = Genes[StepAmount] #Saving the genes but not epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])  
                print("DEBUG ME")
            end
        end
                    
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution_No_Epi, σ_Genes,PositionArray[1,:,:],tspan,Parameters)
    else
        prob = ODEProblem(Epigenetic_Evolution_No_Epi,PositionArray[1,:,:],tspan,Parameters)
    end    #Solving

    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        sol = solve(prob,LambaEulerHeun())
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray[1,:,:] = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])  
        print("DEBUG ME")

    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    
    return PositionArray[2,:,:],Random_Noise, Orbit_Collection,PositionArray[1,:,:]
end

function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit,Selected_Amount)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Selected_Amount         = How many cells we will select for mutation to the next generation
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:Selected_Amount]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:Selected_Amount]
    Indices = Int.(Indices)
    #print(Indices[2,1:Selected_Amount])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit
end


In [ ]:
SplitTime::Float32 = 20 #Time before a splitting event happens
N::Int32 = 32 #How many copies of same network we have
Splits::Int32 = 3 #How often will our cell split
M::Int32 = 100 #This is the amount of genes we want in our network
#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!

#Constant and plotting tools
a::Int32 = 1.0   #Ratio of gene to epigenetic factor
InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Counter = 0 #Used to randomise the mutations, leave to keep same seed
Perturb_Counter = 0 #For the initial condition perturbations, leave to keep same perturbations


Evolutionary_Complexity = []
CArray_Collection = zeros(Float32,N,M)
PositionArray_Collection = zeros(Float32,N,2,1,M)
ParameterMatrix_Collection = zeros(Int32,N,M,M)
Final_Epigenetic_Factors = zeros(Float32,N,2^(Splits),M)
Final_Regular_Factors = zeros(Float32,N,2^(Splits),M)

Sparse = true
#[110, 108, 71, 16, 123, 119, 116, 103]
for i=1:N
    RandomInitial = 400 + 100 #Seed for Initial conditions

    CArray_Collection[i,:], PositionArray_Collection[i,:,:,:], ParameterMatrix_Collection[i,:,:] = Setting_Up_Randomly(M,Sparse,MersenneTwister(RandomInitial))
    #Sets the initial conditions that we will be reusing in the simulation (Change RandomInitial for different values)
    
end

Initial_Unperturbed = deepcopy(PositionArray_Collection)
Random_Noise = 574 #seed for splitting noise

#Need to define outside for loop, otherwise it will be destroyed afterwards
Times = 0
Genes = 0


#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = true
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
KArray[1:10] .= 1 #Here we can manually adjust which genes will have diffusion
#Noise parameters
Splitting_Strength = 0.05f0
Stochastic_Noise = false
Noise = 0.001
v::Float32 = 6*10^-4  #Speed at which epigenetic factor changes
#v::Float32 = 0  #Speed at which epigenetic factor changes


In [ ]:
K=4
Cell_Bit = []
Selected_Amount = Int.(N/4)
Generations = 200
Means = zeros(Generations)
Mean_Std = zeros(Generations)
Mutate_Counter = 1
Data_Name = "Diffusing_Network_With_Epi"
Pathway = "Data_All/" * Data_Name
mkpath(Pathway)


@showprogress 1 for Gen =1:Generations
    for x=1:N
        if v < -0.000001
            Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits,Final_Regular_Factors[x,:,:] = Running_Network_No_Epi(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)

        else
            Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits,Final_Regular_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)

        end
    end
    #Indices = [i for i=1:8]
    if v < 0.000001
        Indices,AllIndices,Evolutionary_Complexity,Cell_Collection, Cell_Bit = Scoring_Indexes(N,Final_Regular_Factors,Cell_Bit,Selected_Amount)
        display(Epi_Plotter(Indices,K,Final_Regular_Factors,Gen))

    else
        Indices,AllIndices,Evolutionary_Complexity,Cell_Collection, Cell_Bit = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit,Selected_Amount)
        display(Epi_Plotter(Indices,K,Final_Epigenetic_Factors,Gen))

    end


    #Now we mutate the best performing networks
    Mutated_ParameterMatrix_Collection = zeros(Int32,N,M,M)
    MutateCounter = 1
    for p=1:Selected_Amount
        for pau=1:4
            Mutated_Network = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,1,MutateCounter)
            MutateCounter +=1
            Mutated_ParameterMatrix_Collection[(p-1)*4+pau,:,:] = Mutated_Network
        end
    end
    ParameterMatrix_Collection = deepcopy(Mutated_ParameterMatrix_Collection)
    Means[Gen] = mean(Evolutionary_Complexity./2^Splits)
    Mean_Std[Gen] = std(Evolutionary_Complexity./2^Splits)
    if 0 == mod(Gen,10) #We save our score and network every 10 steps
        Generation_Plot_Update = plot(Means[1:Gen],yerr = Mean_Std[1:Gen],lc=:blue,msc=:blue)
        display(Generation_Plot_Update)
        savefig(Generation_Plot_Update,Pathway * "/FinalScore.png")

        Location1 = Pathway * "/Gen:"*string(Gen)*"_1.jld"
        save(Location1, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterMatrix_Collection[Indices[1],:,:])

        Location2 = Pathway * "/Gen:"*string(Gen)*"_2.jld"
        save(Location2, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterMatrix_Collection[Indices[2],:,:])


    end
end

In [ ]:
plot(Means,yerr = Mean_Std,lc=:blue,msc=:blue)


In [ ]:
Previous_Gens = length(Means)


In [ ]:
Previous_Gens = length(Means)
New_Gens = 300
Means_New = zeros(Previous_Gens+New_Gens)
Mean_Std_New = zeros(Previous_Gens+New_Gens)
Means_New[1:Previous_Gens] = Means
Mean_Std_New[1:Previous_Gens] = Mean_Std
Means = Means_New
Mean_Std = Mean_Std_New
@showprogress 1 for Gen =1:New_Gens
    Gen = Gen+Previous_Gens
    for x=1:N
        Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits,Final_Regular_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
    end
    #Indices = [i for i=1:8]
    Indices,AllIndices,Evolutionary_Complexity,Cell_Collection, Cell_Bit = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit,Selected_Amount)
    display(Epi_Plotter(Indices,K,Final_Epigenetic_Factors,Gen))


    #Now we mutate the best performing networks
    Mutated_ParameterMatrix_Collection = zeros(Int32,N,M,M)
    MutateCounter = 1
    for p=1:Selected_Amount
        for pau=1:4
            Mutated_Network = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,1,MutateCounter)
            MutateCounter +=1
            Mutated_ParameterMatrix_Collection[(p-1)*4+pau,:,:] = Mutated_Network
        end
    end
    ParameterMatrix_Collection = deepcopy(Mutated_ParameterMatrix_Collection)
    Means[Gen] = mean(Evolutionary_Complexity./2^Splits)
    Mean_Std[Gen] = std(Evolutionary_Complexity./2^Splits)
    if 0 == mod(Gen,10) #We save our score and network every 10 steps
        Generation_Plot_Update = plot(Means[1:Gen],yerr = Mean_Std[1:Gen],lc=:blue,msc=:blue)
        display(Generation_Plot_Update)
        savefig(Generation_Plot_Update,Pathway * "/FinalScore.png")

        Location1 = Pathway * "/Gen:"*string(Gen)*"_1.jld"
        save(Location1, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterMatrix_Collection[Indices[1],:,:])



    end
end

In [ ]:
Previous_Gens = 400
New_Gens = 300
Means_New = zeros(Previous_Gens+New_Gens)
Mean_Std_New = zeros(Previous_Gens+New_Gens)
Means_New[1:Previous_Gens] = Means[1:400]
Mean_Std_New[1:Previous_Gens] = Mean_Std[1:400]
Means = Means_New
Mean_Std = Mean_Std_New
@showprogress 1 for Gen =1:New_Gens
    Gen = Gen+Previous_Gens
    for x=1:N
        Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits,Final_Regular_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
    end
    #Indices = [i for i=1:8]
    Indices,AllIndices,Evolutionary_Complexity,Cell_Collection, Cell_Bit = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit,Selected_Amount)
    display(Epi_Plotter(Indices,K,Final_Epigenetic_Factors,Gen))


    #Now we mutate the best performing networks
    Mutated_ParameterMatrix_Collection = zeros(Int32,N,M,M)
    MutateCounter = 1
    for p=1:Selected_Amount
        for pau=1:4
            Mutated_Network = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,1,MutateCounter)
            MutateCounter +=1
            Mutated_ParameterMatrix_Collection[(p-1)*4+pau,:,:] = Mutated_Network
        end
    end
    ParameterMatrix_Collection = deepcopy(Mutated_ParameterMatrix_Collection)
    Means[Gen] = mean(Evolutionary_Complexity./2^Splits)
    Mean_Std[Gen] = std(Evolutionary_Complexity./2^Splits)
    if 0 == mod(Gen,10) #We save our score and network every 10 steps
        Generation_Plot_Update = plot(Means[1:Gen],yerr = Mean_Std[1:Gen],lc=:blue,msc=:blue)
        display(Generation_Plot_Update)
        savefig(Generation_Plot_Update,Pathway * "/FinalScore.png")

        Location1 = Pathway * "/Gen:"*string(Gen)*"_1.jld"
        save(Location1, "CArray", CArray_Collection[1,:] , "PositionArray", PositionArray_Collection[1,1,:,:],"Network",ParameterMatrix_Collection[Indices[1],:,:])



    end
end

In [ ]:
display(plot(Means4,yerr = Mean_Std4,lc=:blue,msc=:blue))
display(plot(Means4,yerr = Mean_Std4,lc=:blue,msc=:blue))


In [ ]:
Temporary_Orbits = []
for x=1:N
    Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits,Final_Regular_Factors[x,:,:] = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
    print(size(Final_Epigenetic_Factors[x,:,:]))
end
    

In [ ]:
Temporary_Orbits = []
for x=1:N
    Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
    print(size(Final_Epigenetic_Factors[x,:,:]))
end


In [ ]:
display(plot([i for i=1:16],Final_Epigenetic_Factors[1,:,:]))

for j=2:N
    display(plot([i for i=1:16],Final_Epigenetic_Factors[j,:,:]))

end


In [ ]:
Splitting_Strength = 0.05f0
Stochastic_Noise = true
Diffusing_System = false

Noise = 0.001
Temporary_Orbits = []
for x=1:N
    Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
    print(size(Final_Epigenetic_Factors[x,:,:]))
end
display(plot([i for i=1:16],Final_Epigenetic_Factors[1,:,:]))

for j=2:N
    display(plot([i for i=1:16],Final_Epigenetic_Factors[j,:,:]))

end

In [ ]:
Splitting_Strength = 0.00f0
Stochastic_Noise = true
Diffusing_System = false

Noise = 0.001
Temporary_Orbits = []
for x=1:N
    Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
    print(size(Final_Epigenetic_Factors[x,:,:]))
end
display(plot([i for i=1:16],Final_Epigenetic_Factors[1,:,:]))

for j=2:N
    display(plot([i for i=1:16],Final_Epigenetic_Factors[j,:,:]))

end

In [ ]:
Splitting_Strength = 0.00f0
Stochastic_Noise = true
Diffusing_System = false

Noise = 0.0001
Temporary_Orbits = []
for x=1:N
    Final_Epigenetic_Factors[x,:,:],Random_Noise,Temporary_Orbits = Running_Network(CArray_Collection[x,:],ParameterMatrix_Collection[x,:,:],PositionArray_Collection[x,:,:,:],Random_Noise)
    print(size(Final_Epigenetic_Factors[x,:,:]))
end
display(plot([i for i=1:16],Final_Epigenetic_Factors[1,:,:]))

for j=2:N
    display(plot([i for i=1:16],Final_Epigenetic_Factors[j,:,:]))

end